In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('data/che.csv')

In [5]:
df.head()

,CompanyNo,CompanyName,che,label
0,1,Rampart Division,킹고인 창업경진대회 <장려상> 성균관대학교 캠퍼스타운\n창업아이템 검증프로그램 <선...,B
1,8,술며든다,초기창업패키지 킹고창업클럽\n제4차 심화형 일반인 창업강좌\n포스코 창업 인큐베이팅...,C
2,12,라이더타운,"창업의 이해\n디자인 씽킹 실습\n창업 아이템 구체화, 고객반응 확인\n스타트업 시...",B
3,13,FIT ME,제 13특수임무여단 부중대장 특전사 임무 수행\nYOUFIT 매니저 회원 운동처방 ...,A
4,53,지중해,스타벅스 창업카페 -인사이트 공유 프로그램- 초기창업 시 주의점,C


In [13]:
import gluonnlp as nlp
from torch.utils.data import Dataset, DataLoader 
import torch.nn as nn 
import torch 
import numpy as np 
import pandas as pd 
import glob 

from utils import * 
from settings import *
import argparse, os
import numpy as np 

import torch 
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

from transformers import AdamW

import gluonnlp as nlp
from kobert import get_pytorch_kobert_model, get_tokenizer
from sklearn.model_selection import train_test_split

from utils import * 
from settings import * 
from bert_utils import * 

In [14]:
import gluonnlp as nlp
from torch.utils.data import Dataset, DataLoader 
import torch.nn as nn 
import torch 
import numpy as np 
import pandas as pd 
import glob 

from utils import * 
from settings import *

class BERTDataset(Dataset):
    def __init__(self, X:pd.Series, Y:pd.Series, tokenizer, max_len, pad=True, pair=False):
        '''
        Parameters
        ----------
        X: sentences
        Y: label 


        df : dataframe (trainset or testset)
        X : sentences -> str
        Y : label -> int
        '''
        self.tokenizer = tokenizer 
        self.max_len = max_len 
        self.pad = pad 
        self.pair = pair

        self.transform = nlp.data.BERTSentenceTransform(tokenizer, max_seq_length=self.max_len, pad=self.pad, pair=self.pair)

        self.X = X # sentence
        self.Y = Y # label
        self.sentences = [self.transform(x) for x in self.X]
        self.labels = [np.int8(y) for y in self.Y]

    def __getitem__(self, i):
        return self.sentences[i] + (self.labels[i], )

    def __len__(self):
        return len(self.labels)


class BERTClassifier(nn.Module):
    def __init__(self, model, hidden_size, num_classes, dr_rate):
        super(BERTClassifier, self).__init__()
        self.model = model 
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(self.hidden_size, self.num_classes)

        if self.dr_rate: 
            self.dropout = nn.Dropout(p=self.dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):

        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        _, pooler = self.model(input_ids=token_ids, token_type_ids=segment_ids, attention_mask=attention_mask.float())
        
        if self.dr_rate: 
            pooler = self.dropout(pooler)

        output = self.classifier(pooler)
        return output



def get_file_info(get_file_name = False):
    '''
    Channels.csv
    CostStructure.csv
    CustomerRelationships.csv
    CustomerSegmentations.csv
    KeyActivities.csv
    KeyPartners.csv
    KeyResources.csv 
    RevenueStreams.csv
    ValuePropositions.csv
    '''
    file_path = glob.glob(f'{BASE_DIR}/data/*.csv') # concatenate dataset (it, manufacture)
    return sorted(file_path)

def label_encoder(data, reverse=False):
    label_dic = {
        'A' : 0,
        'B' : 1, 
        'C' : 2
    }
    if reverse:
        label_dic = {value: key for key, value in label_dic.items()}
    return label_dic[data]



def preprocessing(file_path):
    # label encoding [ e.g. A -> 0, B -> 1]
    df = pd.read_csv(file_path)
    
    df.iloc[:, -1] = df.iloc[:, -1].apply(lambda x: label_encoder(x) if x in ['A', 'B', 'C'] else np.NaN)
    df.dropna(inplace=True, axis=0)

    return df

In [21]:
df.iloc[:, -1] = df.iloc[:, -1].apply(lambda x: label_encoder(x) if x in ['A', 'B', 'C'] else np.NaN)
df.dropna(inplace=True, axis=0)


/tmp/ipykernel_82347/3383164954.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, -1] = df.iloc[:, -1].apply(lambda x: label_encoder(x) if x in ['A', 'B', 'C'] else np.NaN)


In [17]:
bert_models, vocab = get_pytorch_kobert_model()
bert_tokenizer = get_tokenizer()
tokenizer = nlp.data.BERTSPTokenizer(bert_tokenizer, vocab, lower=False)


using cached model. /home/bax/문서/20221212test/.cache/kobert_v1.zip
using cached model. /home/bax/문서/20221212test/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/bax/문서/20221212test/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [19]:
max_len=1028
batch_size=128

In [22]:
train = BERTDataset(df.iloc[:, -2], df.iloc[:, -1], tokenizer=tokenizer, max_len=max_len)
train_loader = DataLoader(train, batch_size, shuffle=False, drop_last=False)

In [23]:
train_loader